# 对象引用、可变性和垃圾回收

本章讨论对象标识、值和别名等概念，揭露元组的一个神奇特性：元组是不可变的，但是其中的值可以改变，引申
到浅复制和深复制，最后一节讨论垃圾回收、`del`命令。

## 变量不是盒子

Python 变量类似于 Java 中的引用式变量，把它们理解为附加在对象上的标注

In [1]:
# 变量a与b引用同一个变量
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

对引用式变量来说，说把变量分配给对象更合理，反过来说就有问题。

## 标识、相等性和别名

- 每个变量都有标识、类型和值。
- 对象一旦创建，它的标识绝不会变；你可以把标识理解为对象在内存中的地址。
- `is` 运算符比较两个对象的标识； `id()` 函数返回对象标识的整数表示。

对象 ID 的真正意义在不同的实现中有所不同。
在 CPython 中， id() 返回对象的内存地址，但是在其他 Python 解释器中可能是别的值。
关键是， ID 一定是唯一的数值标注，而且在对象的生命周期中绝不会变。

标识最常使用 is 运算符检查，而不是直接比较 ID。

相同不可变对象的`id`也未必相同，这取决语言实现的[细节](https://www.zhihu.com/question/25050656)

### 在==和is之间选择

`==` 运算符比较两个对象的值（对象中保存的数据），而 `is` 比较对象的标识。

在变量和单例值之间比较时，应该使用 `is`
```python3
# 检查变量绑定的值是不是 None
x is None
# 否定
x is not None
```

- `is` 运算符速度快, 不能重载，不用寻找并调用特殊方法，而是直接比较两个整数 ID
- `==` 是语法糖，等同于`__eq__`。 object 的 `__eq__`方法比较两个对象的ID，继承的时候可能被覆盖

### 元组的相对不可变性

元组与多数 Python 集合（列表、字典、集，等等）一样，保存的是对象的引用。
如果引用的元素是可变的，即便元组本身不可变，元素依然可变。

>元组的值会随着引用的可变对象的变化而变。元组中不
可变的是元素的标识。

## 默认浅复制

In [2]:
l1 = [1, [2, 3], (4,5,6)]
l2 = list(l1)
l2

[1, [2, 3], (4, 5, 6)]

In [3]:
l2 == l1

True

In [4]:
l2 is l1

False

In [5]:
# 第三个元素相同
l2[2] is l1[2]

True

构造方法或 [:] 做的是浅复制（副本中的元素是源容器中元素的引用)

演示 `copy()` 和 `deepcopy()` 的用法

In [6]:
class Bus:
    def __init__(self, passengers=None):
        self.passengers = [] if passengers is None else list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [7]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(140393428602552, 140393428602608, 140393428602832)

In [8]:
bus1.drop('Bill')
# 浅复制
bus2.passengers

['Alice', 'Claire', 'David']

In [9]:
# 深复制
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [10]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(140393428634120, 140393428634120, 140393428208200)

>Deep copy is a process in which the copying process occurs recursively. It means first constructing a new collection object and then recursively populating it with copies of the child objects found in the original.

## 函数的参数作为引用时

Python 的参数传递模式是**共享传参（call by sharing**

共享传参指函数的各个形式参数获得实参中各个引用的副本。
也就是说，函数内部的形参是实参的别名。

函数可能会修改作为参数传入的可变对象，但是无法修改那些对象的标识。

### 不要使用可变类型作为参数的默认值

使用可变的对象作为参数的默认值会产生比较隐蔽的问题

In [11]:
class HauntedBus:
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [12]:
bus1 = HauntedBus()
bus1.pick('Alice')
bus2 = HauntedBus()
bus2.passengers
# 由于引用默认列表，bus1和bu2共享乘客

['Alice']

In [13]:
id(HauntedBus.__init__.__defaults__[0])

140393429103688

In [14]:
id(bus2.passengers)

140393429103688

问题在于，`functions.__defaults__`是一个tuple，即函数签名中的默认引用。
该tuple在函数定义的时候确定，每个使用默认参数的函数对象共享该参数。

### 防御可变参数

如果定义的函数接收可变参数，应该谨慎考虑调用方是否期望修改传入的参数。

可以采取:

- 保证参数内容不改变
- 深复制

## del和垃圾回收

del 仅删除变量，而不是对象。删除最后一个保存对象引用的变量，导致对象被销毁。

CPython 中，垃圾回收使用的主要算法是引用计数。

## 弱引用

有时需要引用一个对象，但不想增加引用计数，从而影响对象被回收。

弱引用在缓存应用中很有用，因为我们不想仅因为被缓存引用着而始终保存缓存对象。

### eakValueDictionary简介  弱引用的局限

In [41]:
#todo
